In [1]:
import time
import jsonlines
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def scrape_quizlet(quizlet_url, output_file):

    driver = webdriver.Chrome()

    try:
        driver.get(quizlet_url)
        
        time.sleep(5)
        
        terms = []
        definitions = []
        
        term_elements = driver.find_elements(By.CLASS_NAME, 'SetPageTerms-term')
        x = True

        for term_elem in term_elements:
            print(term_elem.text.strip())
            # if(x):
            #     terms.append(term_elem.text.strip())
            #     x = False
            # else:
            #     definitions.append(term_elem.text.strip())
            #     x = True

            text_elements = term_elem.find_elements(By.CLASS_NAME, 'TermText notranslate lang-en')
            print(text_elements + "ASASAS")
            if len(text_elements) >= 2:
                term = text_elements[0].text.strip()
                definition = text_elements[1].text.strip()
                terms.append(term)
                definitions.append(definition)
            else:
                print(f"Skipping element: {term_elem.text.strip()}")

        
        with jsonlines.open(output_file, mode='w') as writer:
            for term, definition in zip(terms, definitions):
                data = {'term': term, 'definition': definition}
                writer.write(data)
        
        print(f"Scraping completed. Data saved to {output_file}")
    
    except Exception as e:
        print(f"Error occurred during scraping: {str(e)}")
    
    # finally:
    #     driver.quit()


quizlet_url = 'https://quizlet.com/847731671/ap-music-theory-flash-cards/'
output_file = 'quizlet_data.jsonl'
scrape_quizlet(quizlet_url, output_file)
